# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

In [1]:
import json
import pandas as pd

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [2]:
print(f"[Task 1.1] Answer:\n")
with open("./data/contributors_sample.json", "r") as file:
    data = json.load(file)

print(f"First user: \n{data[0]}\n\nSecond user: \n{data[1]}\n\nThird user: \n{data[2]}\n")
print(f"Three users: \n{pd.DataFrame(data[:3])}") # pandas

[Task 1.1] Answer:

First user: 
{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}

Second user: 
{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}

Third user: 
{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}

Three users: 
      username            name sex  \
0      uhebert  Lindsey Nguyen   F   
1  vickitaylor    Chery

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [3]:
print(f"[Task 1.2] Answer:\n")

domainList = []

for i in data:
    domainList.append(i['mail'].split("@")[1])  
    
print(set(domainList))

[Task 1.2] Answer:

{'gmail.com', 'hotmail.com', 'yahoo.com'}


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [4]:
print(f"[Task 1.3] Answer:\n")

def userInfoByName(userName):
    for user in data:
        if user['username'] == userName:
            return user
    raise ValueError(f"{userName} is not on the list!")

print(f"Valid: \n\n{userInfoByName('vickitaylor')} \n\nInvalid: ")
print(f"{userInfoByName('farkhad')}")

[Task 1.3] Answer:

Valid: 

{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970} 

Invalid: 


ValueError: farkhad is not on the list!

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [5]:
print(f"[Task 1.4] Answer:\n")

cntF = 0
cntM = 0

for i in data:
    if i['sex'] == "F":
        cntF += 1
    else:
        cntM +=1


print(f"Number of users: {len(data)}\nNumber of males: {cntM}\nNumber of females: {cntF}")

[Task 1.4] Answer:

Number of users: 4200
Number of males: 2064
Number of females: 2136


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [6]:
print(f"[Task 1.5] Answer:\n")

dataList = []

for user in data:
    dataList.append({"id": user['id'], "username": user['username'], "sex": user['sex']})

contributors = pd.DataFrame.from_dict(dataList)
print(contributors)

[Task 1.5] Answer:

           id       username sex
0       35193        uhebert   F
1       91970    vickitaylor   F
2     1848091    sheilaadams   F
3       50969       nicole82   F
4      676820         jean67   M
...       ...            ...  ..
4195   423555  stevenspencer   F
4196    35251      rwilliams   M
4197   135887      lmartinez   F
4198   212714     brendahill   M
4199   344321       mistyray   F

[4200 rows x 3 columns]


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [7]:
print(f"[Task 1.6] Answer:\n")

recipes = pd.read_csv("./data/recipes_sample.csv", sep=",")
# print(f"Recipes data: {recipes}")

merge = pd.merge(recipes, contributors, left_on=['contributor_id'], right_on=['id'], how="left")
groupby = merge.groupby("contributor_id")["username"].count()
result = groupby[groupby == 0]
print(f"Number of users without info: {result.shape[0]}")

[Task 1.6] Answer:

Number of users without info: 4204


### pickle

In [8]:
import pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [9]:
print(f"[Task 2.1] Answer:\n")

with open('data/contributors_sample.json', 'r') as file:
    data2 = json.load(file)

res = {}
for user in data2:
    for job in user["jobs"]:
        if job not in res:
            res[job] = []
        res[job].append(user["username"])
 
# print(f"{res}")

[Task 2.1] Answer:



2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [10]:
print(f"[Task 2.2] Answer:\n")

with open ('job_people.pickle', 'wb') as file:      # wb - write binary
    pickle.dump(res, file)

with open ('job_people.json', 'w') as file:
    json.dump(res, file, indent='\t')               # indent - for pretty-print

[Task 2.2] Answer:



2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [13]:
print(f"[Task 2.3] Answer:\n")

with open ('job_people.pickle', 'rb') as file:
    rb = pickle.load(file)

# rb

[Task 2.3] Answer:



### XML

In [18]:
from bs4 import BeautifulSoup
import xml

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [20]:
print(f"[Task 3.1] Answer:\n")

soup = BeautifulSoup('data/steps_sample.xml', 'xml')
soup

[Task 3.1] Answer:



FeatureNotFound: Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
print(f"[Task 3.2] Answer:\n")

[Task 3.2] Answer:



3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
print(f"[Task 3.3] Answer:\n")

[Task 3.3] Answer:



3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
print(f"[Task 3.4] Answer:\n")

[Task 3.4] Answer:



3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
print(f"[Task 3.5] Answer:\n")

[Task 3.5] Answer:

